In [ ]:
def lense_json(
    val: TPyJSON, lens: Lens = None
) -> Dict[Lens, ot_types.AttributeValue]:
    """Convert a json structure to an OpenTelemetry attributes dict.

    OT dicts have limited types in their values so a single JSON may need to be
    converted to multiple OT key/value pairs. The approach is to store
    paths/lenses into basic JSON elements, serialized into strings, as keys and
    those basic elements as values.
    """

    if lens is None:
        lens = Lens()

    if isinstance(val, TPyJSONBase):
        return {lens: val}

    if isinstance(val, Dict):
        ret = {}
        for k, v in val.items():
            sublens = lens[k]
            ret.update(lense_json(v, sublens))
        return ret

    elif isinstance(val, Sequence):
        if len(val) == 0:
                return {lens: val}
        typ = type(val[0])

        if isinstance(val[0], TPyJSON) and all(isinstance(x, typ) for x in val):
            return {lens: val}
        
        ret = {}
        for i in range(len(val)):
            sublens = lens[i]
            subval = val[i]
            ret.update(lense_json(subval, sublens))

        return ret

    else:
        raise ValueError(f"Unexpected type: {type(val)}")

lense_json({"hello": 1, "true": 2})

In [ ]:
from typing import Union, Dict, List, Mapping, Sequence, TypeVar, Generic, Optional

TPyJSONBase = Union[str, int, float, bool]

TPyJSON = Union[TPyJSONBase, Mapping[str, 'TPyJSON'], Sequence['TPyJSON']]
from opentelemetry.util import types as ot_types

"""
AttributeValue = Union[
    str,
    bool,
    int,
    float,
    Sequence[str],
    Sequence[bool],
    Sequence[int],
    Sequence[float],
]
"""

from trulens_eval.utils.serial import Lens

T = TypeVar("T")

class LensMappedDict(Generic[T], Dict[str, T]):
    def __init__(self, store: Optional[dict] = None, lens: Lens = None):
        if store is None:
            store = {}
        self.store = store

        if lens is None:
            lens = Lens() 
        self.lens = lens

        self.relative_store = {}

    def __repr__(self):
        return repr(self.relative_store)

    def __str__(self):
        return str(self.relative_store)

    def unmap_value(self, value: Union[T, 'LensMappedDict[T]']) -> T:
        if isinstance(value, LensMappedDict):
            ret = {}
            for k, v in value.relative_store.items():
                ret[k] = value.unmap_value(v)

            return ret
            
        elif isinstance(value, TPyJSONBase):
            return value

        elif value is None:
            return value

        else:
            raise ValueError(f"Unexpected type: {type(value)}")

    def map_value(self, lens: Lens, value: T) -> Union[T, 'LensMappedDict[T]']:
        if isinstance(value, TPyJSONBase):
            return value

        elif isinstance(value, Dict):
            temp = LensMappedDict(store=self.store, lens=lens)
            for k, v in value.items():
                temp[k] = v
            return temp

        elif isinstance(value, Sequence):
            if len(value) == 0:
                return value
            typ = type(value[0])

            if isinstance(value[0], TPyJSON) and all(isinstance(x, typ) for x in value):
                return value

            temp = LensMappedDict(store=self.store, lens=lens)
            for i in range(len(value)):
                temp[i] = value[i]

            return temp

        else:
            raise ValueError(f"Unexpected type: {type(value)}")

    def __getitem__(self, key: str, default: Optional[T] = None) -> T:
        return self.relative_store.get(key, default)

    def __setitem__(self, key: str, value: T) -> None:
        lens = self.lens[key]
        mapped_val = self.map_value(lens, value)

        if key in self.relative_store:
            # Need to delete as there might be multiple store lenses associated
            # with this key.
            del self[key]
        
        self.store[lens] = mapped_val
        self.relative_store[key] = mapped_val

    def __delitem__(self, key: str) -> None:
        lens = self.lens[key]
        val = self.relative_store[key]

        if isinstance(val, LensMappedDict):
            del val

        del self.store[lens]
        del self.relative_store[key]

    def __del__(self):
        for k, v in list(self.relative_store.items()):
            sublens = self.lens[k]

            if hasattr(v, "__del__"):
                del v

            del self.relative_store[k]
            del self.store[sublens]



In [ ]:
temp = LensMappedDict()

In [ ]:
temp['something'] = {'a': 1, 'b': 2, 'sub': {'c': 3, 'd': 4}}
temp.store

In [ ]:
temp['something']['sub'] = 42
temp.store

In [ ]:
type(temp.relative_store['something'])

# Working with Spans

In [ ]:
%load_ext autoreload
%autoreload 2
from pathlib import Path
import sys

# If running from github repo, can use this:
repo = Path().cwd().parent.parent.resolve()
sys.path.append(str(repo))

In [ ]:
from pprint import pformat
from pprint import pprint

from examples.expositional.end2end_apps.custom_app.custom_app import CustomApp
from examples.expositional.end2end_apps.custom_app.custom_retriever import CustomRetriever
import pandas as pd

from trulens_eval import instruments
from trulens_eval.trace.category import Categorizer
from trulens_eval.tru_custom_app import TruCustomApp

In [ ]:
from trulens_eval import Tru
Tru().reset_database()
Tru().start_dashboard(_dev=repo, force=True)

In [ ]:
# Create custom app:
ca = CustomApp(delay=0.0, alloc=0)

# Create trulens wrapper:
ta = TruCustomApp(
    ca,
    app_id="customapp",
)

In [ ]:
instruments.Instrument().print_instrumentation()

In [ ]:
ta.print_instrumented()

In [ ]:
with ta as recorder:
    res = ca.respond_to_query(f"hello")

rec = recorder.get()

In [ ]:
rec.calls[0].model_dump()

In [ ]:
spans = Categorizer.spans_of_record(rec)

pd.DataFrame(
    [(
        s.trace_id & 0xff,
        s.name,
        type(s),
        s.span_type,
        s.span_id & 0xff,
        s.parent_span_id & 0xff if s.parent_span_id else 0,
        s.attributes
    ) for s in spans],
    columns=[
        "trace_id",
        "name",
        "type",
        "span_type",
        "span_id",
        "parent_span_id",
        "attributes"
    ],
)


In [ ]:
for span in spans:
    pprint(span)
    pprint(span.model_dump())
    print()